# Collect review data from Productreview.com.au based on brand name

In [10]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
brands = pd.read_csv('brand_categories.csv',header=None)

In [11]:
brands['brand'] = brands[0].str.lower()
brands['brand'] = brands['brand'].str.replace(' ','-')
brands['brand'] = brands['brand'].str.replace('.','-')
brands['brand'] = brands['brand'].str.replace('--','-')
brands['brand'] = brands['brand'].str.replace("'",'-')
brands['brand'] = brands['brand'].str.replace(",",'-')
brands['brand'] = brands['brand'].str.replace('--','-')
brands

,0,brand
0,Azimo,azimo
1,Raiz Insure,raiz-insure
2,Surveys,surveys
3,iSelf Health Insurance,iself-health-insurance
4,"Cinema, Theme Park and Attraction Vouchers",cinema-theme-park-and-attraction-vouchers
...,...,...
384,Tinklers,tinklers
385,TableTopics,tabletopics
386,Country Culture,country-culture
387,Sputnik Vintage,sputnik-vintage


In [28]:
import requests
from lxml import etree
import time

headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'}
base_url = "https://www.productreview.com.au/listings/{}"


brands['category'] = ''
brands['overall_reviews'] = ''
brands['overall_star'] = ''

brands['star5'] = ''
brands['star4'] = ''
brands['star3'] = ''
brands['star2'] = ''
brands['star1'] = ''

brands['value_for_money'] = ''
brands['value_for_money_num'] = ''
brands['product_quality'] = ''
brands['product_quality_num'] = ''
brands['customer_service'] = ''
brands['customer_service_num'] = ''

num_1 = 0
num_2 = 0
for brand in brands['brand']:
    num_2 +=1
    res = requests.get(base_url.format(f'{brand}'),headers=headers)
    tree = etree.HTML(res.text)
    brand_category = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[1]/nav/ol/li[1]/a/@href")
    if len(brand_category)!=0:
        num_1 += 1
        print(num_1,num_2)
        brand_category = brand_category[0].split('/')[2]
        brands.loc[brands['brand']==brand,'category'] = brand_category
        print(brands.loc[brands['brand']==brand,'category'])
    overall_reviews = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[1]/div[1]/div/div/span/text()[1]")
    overall_star = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[1]/div[1]/span/@data-average-rating")
    if len(overall_reviews)!=0:
        brands.loc[brands['brand']==brand,'overall_reviews'] = overall_reviews[0]
    if len(overall_star)!=0:
        brands.loc[brands['brand']==brand,'overall_star'] = overall_star[0]
    star_ind = "/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[1]/div[2]/a[{}]/@href"
    star_num = "/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[1]/div[2]/a[{}]/span[2]/text()[2]"
    for i in ['1','2','3','4','5','6']:
        ind = tree.xpath(star_ind.format(i))
        num = tree.xpath(star_num.format(i))
        if len(ind)!=0 and len(num)!=0:
            ind = ind[0].split('=')[1]
            brands.loc[brands['brand']==brand,'star'+ind] = num[0]
    try:
        brands.loc[brands['brand']==brand,'value_for_money'] = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[3]/div/div/@title")
        brands.loc[brands['brand']==brand,'value_for_money_num'] = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[3]/div/span/text()")
        brands.loc[brands['brand']==brand,'product_quality'] = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[4]/div/div/@title")
        brands.loc[brands['brand']==brand,'product_quality_num'] = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[4]/div/span/text()")
        brands.loc[brands['brand']==brand,'customer_service'] = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[5]/div/div/@title")
        brands.loc[brands['brand']==brand,'customer_service_num'] = tree.xpath("/html/body/div[1]/div[1]/div[1]/div[2]/div/main/div[1]/div[1]/div/div[2]/div[2]/div[5]/div/span/text()")
    except Exception as e:
         pass
    # print(brands.loc[brands['brand']==brand,:])
    time.sleep(1)



1 1
0    money-transfer-services
Name: category, dtype: object
2 8
7    car-hire
Name: category, dtype: object
3 9
8    clothing-shops-brands
Name: category, dtype: object
4 10
9    food-drink-shops
Name: category, dtype: object
5 11
10    clothing-shops-brands
Name: category, dtype: object
6 13
12    clothing-shops-brands
Name: category, dtype: object
7 14
13    clothing-shops-brands
Name: category, dtype: object
8 15
14    beauty-product-retailers
Name: category, dtype: object
9 16
15    travel-agents-booking-service
Name: category, dtype: object
10 33
32    pet-shops
Name: category, dtype: object
11 79
78    home-decor-shops
Name: category, dtype: object
12 101
100    clothing-shops-brands
Name: category, dtype: object
13 187
186    jewellery-shops-brands
Name: category, dtype: object
14 210
209    mother-baby-shops
Name: category, dtype: object
15 250
249    clothing-shops-brands
Name: category, dtype: object
16 276
275    food-drink-shops
Name: category, dtype: object
17 298
297  

In [29]:
brands.to_csv('brands_review.csv',index=False)

In [26]:
num

[]